In [139]:
import pandas as pd
import pulp
from pulp import GLPK
import numpy as np

In [140]:
df = pd.read_csv("Dea_data.csv")
df = df.drop(columns=['Unnamed: 0'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 13 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Company                                         25 non-null     object 
 1   Year                                            25 non-null     int64  
 2   GHG_Intensity                                   25 non-null     float64
 3   Energy_Intensity_per_Sales                      25 non-null     float64
 4   NonRenewable_Energy_Consumption_and_Production  25 non-null     float64
 5   Water_Intensity_Per_Sales                       25 non-null     float64
 6   NonRecycled_Waste_ration                        25 non-null     float64
 7   ESG_Score                                       25 non-null     float64
 8   GHG_1                                           25 non-null     float64
 9   GHG_2                                        

In [141]:
df.replace(0, 0.01, inplace=True)
df.head()

,Company,Year,GHG_Intensity,Energy_Intensity_per_Sales,NonRenewable_Energy_Consumption_and_Production,Water_Intensity_Per_Sales,NonRecycled_Waste_ration,ESG_Score,GHG_1,GHG_2,GHG_3,1_plus_2,1_plus_2_plus_3
0,SONY US Equity,2023,283.35,96.22,89.9,0.010,3.3,5.09,2.72,8.73,250.54,11.45,261.99
1,SONY US Equity,2022,255.60,95.00,95.1,3.392,2.0,5.03,2.63,12.28,227.68,14.92,242.60
2,SONY US Equity,2021,283.36,40.75,94.4,6.784,2.0,4.97,2.36,10.44,226.74,12.80,239.54
3,SONY US Equity,2020,237.57,40.61,95.8,10.176,2.1,4.79,2.41,15.22,190.47,17.63,208.09
4,SONY US Equity,2019,263.10,40.76,98.1,13.568,4.6,4.51,2.48,15.58,216.95,18.05,235.00


In [142]:
df.shape

(25, 13)

In [151]:
inputs_t1 = np.array(df[df['Year'] == 2019][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                        'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
outputs_t1 = np.array(df[df['Year'] == 2019][['GHG_Intensity', 'ESG_Score']])
inputs_t2 = np.array(df[df['Year'] == 2020][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                        'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
outputs_t2 = np.array(df[df['Year'] == 2020][['GHG_Intensity', 'ESG_Score']])
num_dmus = inputs_t1.shape[0]
num_inputs = inputs_t1.shape[1]
num_outputs = outputs_t1.shape[1]

def calculate_efficiency(inputs, outputs, reference_inputs, reference_outputs):
    model = pulp.LpProblem("Efficiency_Calculation", pulp.LpMaximize)
    lambdas = [pulp.LpVariable(f"lambda_{j}", lowBound=0) for j in range(num_dmus)]
    model += pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(num_dmus)])
    for k in range(num_inputs):
        model += pulp.lpSum([lambdas[j] * reference_inputs[j, k] for j in range(num_dmus)]) <= inputs[i, k]
    model += pulp.lpSum(lambdas) == 1
    model.solve()
        
    if model.status == pulp.LpStatusOptimal:
        efficiency_score = pulp.value(pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(num_dmus)])) / outputs[i, 0]
    else:
        #efficiency_score = None
        efficiency_score = 1
        
    return efficiency_score

In [152]:
def return_periods(year_):
    inputs_t1 = np.array(df[df['Year'] == year_][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                        'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
    outputs_t1 = np.array(df[df['Year'] == year_][['GHG_Intensity', 'ESG_Score']])
    inputs_t2 = np.array(df[df['Year'] == (year_+1)][['Energy_Intensity_per_Sales', 'NonRenewable_Energy_Consumption_and_Production',
                        'Water_Intensity_Per_Sales', 'NonRecycled_Waste_ration']])
    outputs_t2 = np.array(df[df['Year'] == (year_+1)][['GHG_Intensity', 'ESG_Score']])

    return inputs_t1, outputs_t1, inputs_t2, outputs_t2

In [178]:
result_matrix = pd.DataFrame(columns=['SONY', 'APPL', 'DELL', 'HPE', 'SAMSUNG', 'EorT'])
result_matrix.columns

Index(['SONY', 'APPL', 'DELL', 'HPE', 'SAMSUNG', 'EorT'], dtype='object')

In [154]:
df.Company.unique()

array(['SONY US Equity', 'AAPl US Equity', 'DELL US Equity',
       'HPE US Equity', 'SAMSUNG KS Equity'], dtype=object)

In [76]:

def calculate_efficiency(inputs, outputs, reference_inputs, reference_outputs, num_dmus, num_inputs):
    model = pulp.LpProblem("Efficiency_Calculation", pulp.LpMaximize)
    lambdas = [pulp.LpVariable(f"lambda_{j}", lowBound=0) for j in range(num_dmus)]
    model += pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(num_dmus)])
    for k in range(num_inputs):
        model += pulp.lpSum([lambdas[j] * reference_inputs[j, k] for j in range(num_dmus)]) <= inputs[i, k]
    model += pulp.lpSum(lambdas) == 1
    model.solve()
        
    if model.status == pulp.LpStatusOptimal:
        efficiency_score = pulp.value(pulp.lpSum([lambdas[j] * reference_outputs[j, 0] for j in range(num_dmus)])) / outputs[i, 0]
    else:
        #efficiency_score = None
        efficiency_score = 0.01
        
    return efficiency_score

In [179]:
collection_slices = []
for i in [2019, 2020, 2021, 2022]:
    in_t1, out_t1, in_t2, out_t2 = return_periods(i)

    malmquist_indices = []
    EC_list = []
    TC_list = []
    for i in range(num_dmus):
        eff_t1_t1 = calculate_efficiency(in_t1, out_t1, in_t1, out_t1)
        eff_t1_t2 = calculate_efficiency(in_t1, out_t1, in_t2, out_t2)
        eff_t2_t1 = calculate_efficiency(in_t2, out_t2, in_t1, out_t1)
        eff_t2_t2 = calculate_efficiency(in_t2, out_t2, in_t2, out_t2)
        #print(eff_t1_t1, eff_t1_t2, eff_t2_t1, eff_t2_t2)
        if all(v is not None for v in [eff_t1_t1, eff_t2_t2, eff_t1_t2, eff_t2_t1]):
            EC = eff_t2_t2 / eff_t1_t1 if eff_t1_t1 != 0 else None
            TC = np.sqrt((eff_t1_t2 * eff_t2_t2) / (eff_t1_t1 * eff_t2_t1)) if all(v != 0 for v in [eff_t1_t1, eff_t2_t1]) else None
            #print(EC, TC)
            EC_list.append(EC)
            TC_list.append(TC)
            malmquist_index = EC * TC if EC is not None and TC is not None else None
            malmquist_indices.append(malmquist_index)
        else:
            malmquist_indices.append(None)
    malmquist_indices.append('ETC')
    collection_slices.append(malmquist_indices)
    EC_list.append('EC')
    TC_list.append('TC')
    result_matrix.loc[len(result_matrix)] = malmquist_indices
    result_matrix.loc[len(result_matrix)] = EC_list
    result_matrix.loc[len(result_matrix)] = TC_list


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/yanhanjun/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/9b7e46c81c074923a2ea43f10ac921bf-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/8t/3xrvdszj1g75llc2jb40dnvw0000gn/T/9b7e46c81c074923a2ea43f10ac921bf-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 41 RHS
At line 47 BOUNDS
At line 48 ENDATA
Problem MODEL has 5 rows, 5 columns and 25 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 5 (0) columns and 25 (0) elements
0  Obj -0 Primal inf 0.96715805 (1) Dual inf 3515.256 (5)
0  Obj -0 Primal inf 0.96715805 (1) Dual inf 1.3835202e+11 (5)
2  Obj 263.1
Optimal - objective value 263.1
Optimal objective 263.1 - 2 iterations time 0.002
Option for printingOptions change

In [180]:
collection_slices
result_matrix

,SONY,APPL,DELL,HPE,SAMSUNG,EorT
0,0.975709,0.990075,1.280157,1.000000,2.889949,ETC
1,1.000000,1.000000,1.000000,1.000000,1.711253,EC
2,0.975709,0.990075,1.280157,1.000000,1.688791,TC
3,1.093604,0.928379,1.006101,0.988597,0.893422,ETC
4,1.000000,1.000000,1.000000,1.000000,0.584367,EC
5,1.093604,0.928379,1.006101,0.988597,1.528872,TC
6,1.000000,1.117400,1.047505,0.903791,0.951586,ETC
7,1.000000,1.000000,1.000000,1.000000,1.000000,EC
8,1.000000,1.117400,1.047505,0.903791,0.951586,TC
9,1.000000,0.616341,1.038338,1.141700,1.000000,ETC


In [165]:
collection_slices
result_df = pd.DataFrame(collection_slices, columns=['SONY', 'APPL', 'DELL', 'HPE', 'SAMSUNG'])
result_df

,SONY,APPL,DELL,HPE,SAMSUNG
0,0.975709,0.990075,1.280157,1.000000,2.889949
1,1.093604,0.928379,1.006101,0.988597,0.893422
2,1.000000,1.117400,1.047505,0.903791,0.951586
3,1.000000,0.616341,1.038338,1.141700,1.000000
